### ตัวอย่างการคำนวณ SMA และ EMA

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/SCB.csv', index_col=0)
close = df['Close'] # อ้างถึง columns close จาก dataflame ชื่อ df
#calculate SMA โค้ดการคำนวณ SMA
sma10 = close.rolling(window=10).mean() # .mean หาค่าเฉลี่ยเคลื่อนที่อย่างง่าย 10วัน  โดยใช้ Method rolling และกำหนด windowsize เป็น 10 
# 9 วันแรกออกมาเป็น NAN อัตโนมัติ เพราะข้อมูลไม่ครบ 10

#calculate EMA
# span ก็เหมือน window, side adjust คือ
ema10 = close.ewm(span=10, adjust=False).mean() # .mean หาค่าเฉลี่ยเคลื่อนที่แบบ exponantial 10วัน 
ema10[0:9] = np.nan # 9 วันแรกออกมาเป็น NAN ตราบใดข้อมูลยังไม่ครบไม่ต้องหาค่าเฉลี่ย

df['SMA10'] = sma10 # เพิ่ม SMA10 ทีเ่ป็นคอลัมน์เข้าไปใน df
df['EMA10'] = ema10 # เพิ่ม EMA10 ทีเ่ป็นคอลัมน์เข้าไปใน df
print(df.head(10)) # เพื่อให้เห็นข้อมูล 9 วันแรกว่าเป็น NAN ไหม
df.to_csv('data/newSCB.csv')

            Open   High    Low  Close  Volume  SMA10      EMA10
Date                                                           
1/4/2000   45.50  46.00  45.25  45.75   37461    NaN        NaN
1/5/2000   43.50  44.25  43.00  43.50   27363    NaN        NaN
1/6/2000   43.75  43.75  42.25  42.50   11591    NaN        NaN
1/7/2000   43.00  44.00  42.50  43.75    9750    NaN        NaN
1/10/2000  44.50  45.25  43.75  43.75   17305    NaN        NaN
1/11/2000  44.25  44.50  43.00  43.00   10539    NaN        NaN
1/12/2000  42.75  43.75  42.75  43.75    5430    NaN        NaN
1/13/2000  43.50  44.00  43.25  43.50    4684    NaN        NaN
1/14/2000  44.00  44.50  43.75  44.50    8893    NaN        NaN
1/17/2000  45.25  45.75  45.00  45.00   24458   43.9  44.261005


### การหาสัญญาณซื้อขายแบบ EMA crossover (EMACO)

In [2]:
import pandas as pd
import numpy as np
def EMACO_Sig(close,sp,lp):
    rows = len(close) # ให้ค่าจำนวนแถว ต้องรู้เพราะ จะสร้างสัญญาณซื้อขาย(signal line) จะต้องมีจำนวนวัน/จำนวนแถวที่มี 
    signal = [0] * rows # จำนวนวัน * จำนวนแถวที่มีอยู่ สร้างเป็น vector ที่ทุกแถวมีค่าเป็น 0 หมด 0 คือไม่มีสัญญาณ แปลว่าตอนนี้ column signalเป็น 0 ทั้งหมด
    # ต้องคำนวณค่าเฉลี่ยเคลื่อนที่ 2 เส้น คือ EMA Short period และ EMA Long period
    ema_short = close.ewm(span=sp, adjust=False).mean() # sp ที่กำหนดข้างล่างไว้คือ 10 และส่งค่าให้ sp
    ema_short[0:sp-1] = np.nan # แถวแรกจนถึงแถวที่ sp-1 คือ 9 จะต้องเป็น NAN
    ema_long = close.ewm(span=lp, adjust=False).mean() # lp ที่กำหนดไว้ข้างล่างคือ 50 และส่งค่าให้ lp
    ema_long[0:lp-1] = np.nan # แถวแรกจนถึงแถวที่ lp-1 คือ 49 จะต้องเป็น NAN
    # คำนวนไล่่ไปทีละแถวตั้งแต่ 1 ถึง rows ที่เรา len(close) ไว้
    # แต่ละรอบ/แถวของการลูปจะทำการเปรียบเทียบเงื่อนไข
    # ซื้อเมื่อเส้นระยะสั้นตัดเส้นระยะยาวขึ้น และ ขายเมื่อเส้นระยะสั้นตัดเส้นระยะยาวลง
    for i in range(1,rows):
        # เส้นระยะสั้นเคยอยู่ต่ำกว่าเส้นระยะยาว *ema_short[i-1]<=ema_long[i-1]*
        # และเมื่อเส้นระยะสั้นตัดเส้นระยะยาวขึ้นหรือเส้นระยะสั้นสูงกว่าเส้นระยะยาวในปัจจุบันคือ i *ema_short[i]>ema_long[i]*
        # ทำให้เกิดสัญญาณซื้อ
        if (ema_short[i-1]<=ema_long[i-1]) and (ema_short[i]>ema_long[i]):
            signal[i] = 1
        # เส้นระยะสั้นตัดเส้นระยะยาวลง    
        # แปลว่าเส้นระยะสั้นเคยสูงกว่าหรือเท่ากับ/มากกว่าหรือเท่ากับ period ที่แล้ว แต่ปัจจุบันเส้นระยะสั้นต่ำกว่าหรือน้อยกกว่าเส้นระยะยาวแล้ว
        # ทำให้เกิดสัญญาณขาย
        elif (ema_short[i-1]>=ema_long[i-1]) and (ema_short[i]<ema_long[i]):
            signal[i] = -1
    # ดังนั้น signal(สัญญาณซื้อขาย) ถ้าไม่มีการซื้อการขาย signal[i] = 0
    # signal[i] = 1 เป็นสัญญาณซื้อ
    # signal[i] = -1 เป็นสัญญาณขาย
    return [ema_short,ema_long,signal] # return ไปเป็นรูปแบบของ list
    
df = pd.read_csv('data/SCB.csv') # เรียกมาจาก SCB excel จาก pandas
close = df['Close'] # กำหนดตัวแปร clos จาก df
#Set short period as 10, and set long period as 50
[ema_short, ema_long, signal] = EMACO_Sig(close,10,50) # ส่งราคาปิดไป, ส่ง short period คือ 10วัน, ส่ง long period คือ 50 วัน
df['EMA_10'] = ema_short
df['EMA_50'] = ema_long
df['Signal'] = signal
df.to_csv('data/SCB_EMACO.csv')

In [3]:
# ไม่ตั้ง sp กับ lp
import pandas as pd
import numpy as np
def EMACO_Sig(close):
    rows = len(close)
    signal = [0] * rows
    ema_short = close.ewm(span=10, adjust=False).mean()
    ema_short[0:10-1] = np.nan
    ema_long = close.ewm(span=50, adjust=False).mean()
    ema_long[0:50-1] = np.nan    
    for i in range(1,rows):
        if (ema_short[i-1]<=ema_long[i-1]) and (ema_short[i]>ema_long[i]):
            signal[i] = 1
        elif (ema_short[i-1]>=ema_long[i-1]) and (ema_short[i]<ema_long[i]):
            signal[i] = -1
    return [ema_short,ema_long,signal]
    
df = pd.read_csv('data/SCB.csv')
close = df['Close']
#Set short period as 10, and set long period as 50
[ema_short, ema_long, signal] = EMACO_Sig(close)
df['EMA_10'] = ema_short
df['EMA_50'] = ema_long
df['Signal'] = signal
df.to_csv('data/SCB_EMACO123.csv')

### การหาสัญญาณซื้อขายแบบ MACD กลยุทธ์ที่ 1 (เส้น MACD ตัดเส้น 0)

In [4]:
# ลักษณะเดียวกันกับ EMACO แต่แค่ตัดตรงเส้น 0
import pandas as pd
import numpy as np
def MACD1_Sig(close):
    rows = len(close)
    signal = [0] * rows
    ema_12 = close.ewm(span=12, adjust=False).mean() # ค่าเฉลี่ยเคลื่อนที่ 12 วัน
    ema_12[0:11] = np.nan
    ema_26 = close.ewm(span=26, adjust=False).mean() # ค่าเฉลี่ยเคลื่อนที่ 26 วัน
    ema_26[0:25] = np.nan
    macd = ema_12-ema_26  # เอาเส้นสั้นema12 - เส้นยาวema26 จะได้เส้น macd
    for i in range(1,rows):  # ไม่เริ่มจากแถวแรกเพราะ การตัดการตั้งพิจารณาจาก period ก่อนหน้า แต่ถ้าเริ่มจากแถวแรกจะไม่มีตัวเปรียบเทียบ
        if (macd[i-1]<=0) and (macd[i]>0): # macd ตัดเส้น 0 ขึ้น สัญญาณซื้อ
            signal[i] = 1
        elif (macd[i-1]>=0) and (macd[i]<0): # macd ตัดเส้น 0 ลง สัญญาณขาย
            signal[i] = -1
    return [macd,signal]

df = pd.read_csv('data/SCB.csv')
close = df['Close']
[macd, signal] = MACD1_Sig(close)
df['MACD'] = macd # เอา series macd ไปเพิ่มใน df และตั้งชื่อ column ว่า MACD
df['Signal'] = signal # เอา series signal ไปเพิ่มใน df และตั้งชื่อ column ว่า Signal
df.to_csv('data/SCB_MACD1.csv')

### การหาสัญญาณซื้อขายแบบ MACD กลยุทธ์ที่ 2 (MACD ตัด Signal Line (SL))

In [5]:
import pandas as pd
import numpy as np
def MACD2_Sig(close):
    rows = len(close)
    signal = [0] * rows
    ema_12 = close.ewm(span=12, adjust=False).mean() # เริ่มคำนวณแถวที่ 12 ก่อนหน้านั้นเป็น nan
    ema_12[0:11] = np.nan
    ema_26 = close.ewm(span=26, adjust=False).mean() # เริ่มคำนวณแถวที่ 26 ก่อนหน้านั้นเป็น nan
    ema_26[0:25] = np.nan
    macd = ema_12-ema_26
    sl = macd.ewm(span=9, adjust=False).mean() # คำนวณเส้น sl จาก macd เรา macd มาหาค่าเฉลี่ยเคลื่อนที่แบบ expo 9 วัน
    sl[0:25+9-1] = np.nan # ตั้งแต่แถวแรกจนถึงแถวที่ 33 ที่เริ่มจากแถวที่ 34 เพราะ macd เริ่ม 26 แถวดังนั้นเริ่มคำนวน sl จากแถวที่26 ดังนั้นค่าเฉลี่ยเคลื่อนที่จะเริ่มในอีก 9วันต่อไป
    for i in range(1,rows): # ไม่ได้เริ่มจากแถวแรก เริ่มจากแถวที่ 0 
        if (macd[i-1]<=sl[i-1]) and (macd[i]>sl[i]):
            signal[i] = 1
        elif (macd[i-1]>=sl[i-1]) and (macd[i]<sl[i]):
            signal[i] = -1
    return [macd,sl,signal]

df = pd.read_csv('data/SCB.csv')
close = df['Close']
[macd, sl, signal] = MACD2_Sig(close)
df['MACD'] = macd
df['SL'] = sl
df['Signal'] = signal
df.to_csv('data/SCB_MACD2.csv')